In [164]:
#各年のレース情報取得

import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import pandas
import os

#ディレクトリ作成
os.makedirs("/Users/yamagamihiroki/arima2021/raceInfo/")


#過去20年のレースIDの取得
raceid_list=[]
url = "https://db.netkeiba.com/?pid=race_list&word=%5E%CD%AD%C7%CF%B5%AD%C7%B0"
r=requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")

soup_txt_race=soup.find_all(href=re.compile("/race/20"))
for num in range(20):
    raceid_list.append(soup_txt_race[num].attrs['href'])

#過去20年のレースのデータを取得
for count,i in enumerate(raceid_list):
    url = "https://db.netkeiba.com"+i
    r=requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    soup_span = soup.find_all("span")
    len(soup_span)

    #頭数
    allnum=int((len(soup_span)-8)/3)

    #馬の情報を以下で取得
    soup_txt_l=soup.find_all(class_="txt_l")

    #馬の名前
    name=[]
    for num in range(allnum):
        name.append(soup_txt_l[4*num].contents[1].contents[0])

    #騎手名
    jockey=[]
    for num in range(allnum):
        jockey.append(soup_txt_l[4*num+1].contents[1].contents[0])

    #馬番
    soup_txt_r=soup.find_all(class_="txt_r")
    horse_number=[]
    for num in range(allnum):
        horse_number.append(soup_txt_r[1+5*num].contents[0])

    #走破タイム
    runtime=[] 
    for num in range(allnum):
        try:
            runtime.append(soup_txt_r[2+num*5].contents[0])
        except IndexError:
            runtime.append(None)

    #オッズ
    odds=[]
    for num in range(allnum):
        try:
            odds.append(soup_txt_r[3+5*num].contents[0])
        except IndexError:
            odds.append(None)

    #通過順
    soup_nowrap = soup.find_all("td",nowrap="nowrap",class_=None)
    pas = []
    for num in range(allnum):
        try:
            pas.append(soup_nowrap[3*num].contents[0])
        except IndexError:
            pas.append(None)

    #体重
    weight = []
    for num in range(allnum):
        try:
            weight.append(soup_nowrap[3*num+1].contents[0])
        except IndexError:
            weight.append(None)

    #性齢
    soup_tet_c = soup.find_all("td",nowrap="nowrap",class_="txt_c")
    sex_old = []
    for num in range(allnum):
        sex_old.append(soup_tet_c[6*num].contents[0])

    #斤量
    handi = []
    for num in range(allnum):
        handi.append(soup_tet_c[6*num+1].contents[0])

    #上がり
    last = []
    for num in range(allnum):
        try:
            last.append(soup_tet_c[6*num+3].contents[0].contents[0])
        except IndexError:
            last.append(None)

    #人気
    pop = []
    for num in range(allnum):
        try:
            pop.append(soup_span[3*num+10].contents[0])
        except IndexError:
            pop.append(None)
    
    #データ格納
    houseInfo = [name,jockey,horse_number,runtime,odds,pas,weight,sex_old,handi,last,pop]
    
    #CSV書き出し
    #ファイルパス指定
    year=2020-count    
    filepass1="/Users/yamagamihiroki/arima2021/raceInfo/arima{}_test.csv"
    filepass2=filepass1.format(year)
    with open(filepass2, 'a', newline='',encoding="SHIFT-JIS") as f:
        csv.writer(f).writerows(houseInfo)
    col_num=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
    df=pandas.read_csv(filepass2,encoding="SHIFT-JIS",names=col_num)
    #列名追加
    df_mod=df.rename(index={0:"馬名",1:"騎手名",2:"枠順",3:"走破タイム",4:"オッズ",5:"通過順位",6:"馬体重",7:"性齢",8:"斤量",9:"上がり3ハロン",10:"人気"})
    #ファイル書き出し
    filepass3="/Users/yamagamihiroki/arima2021/raceInfo/arima{}.csv"
    filepass4=filepass3.format(year)
    df_mod.to_csv(filepass4)
    #testファイルの削除
    os.remove(filepass2)
    #間隔
    #time.sleep(1)

In [131]:
#各年の出走馬のデータ取得

import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import pandas
import os

#ディレクトリ作成
os.makedirs("/Users/yamagamihiroki/arima2021/horseInfo/")

#過去20年のレースIDの取得
raceid_list=[]
url = "https://db.netkeiba.com/?pid=race_list&word=%5E%CD%AD%C7%CF%B5%AD%C7%B0"
r=requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")

soup_txt_race=soup.find_all(href=re.compile("/race/20"))
for num in range(20):
    raceid_list.append(soup_txt_race[num].attrs['href'])

#過去20年のレースのデータを取得
for count,i in enumerate(raceid_list):
    url = "https://db.netkeiba.com"+i
    r=requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    soup_span = soup.find_all("span")
    len(soup_span)

    #頭数
    allnum=int((len(soup_span)-8)/3)

    #出走馬のページURL取得
    soup_txt_h =soup.find_all(href=re.compile("/horse/"))
    #出走馬のURL
    horse_url_list=[] 
    for num in range(allnum):
        horse_url_list.append(soup_txt_h[num].attrs['href'])

    for horse_url in horse_url_list:
        #出走馬のデータ取得
        url = "https://db.netkeiba.com"+horse_url
        r=requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        soup_span = soup.find_all("span")

        #過去2年の出走レース取得
        year=2020-count
        racepass1="/race/{}|/race/{}"
        racepass2=racepass1.format(year,year-1)
        soup_txt_race=soup.find_all(href=re.compile("racepass2"))

        #過去3回の出走レース名取得
        race_list=[]
        for num in range(1,4):
            try:
                race_list.append(soup_txt_race[num].contents[0])
            except IndexError:
                race_list.append(None)

        #過去の成績取得
        soup_txt_result=soup.find_all(class_="txt_right")

        #過去3戦の成績
        rank_result=[]
        for num in range(0,3):
            try:
                rank_result.append(soup_txt_result[6+11*num].contents[0])
            except IndexError:
                rank_result.append(None)
        #rank_result


        #過去3戦の人気取得
        pop_result=[]
        for num in range(0,3):
            try:
                pop_result.append(soup_txt_result[5+11*num].contents[0])
            except IndexError:
                pop_result.append(None)
        #pop_result

        #過去レースのオッズ
        odds_result=[]
        for num in range(0,3):
            try:
                odds_result.append(soup_txt_result[4+11*num].contents[0])
            except IndexError:
                odds_result.append(None)
        #odds_result


        #みんなの評価の取得
        soup_txt_review=soup.find_all(src=re.compile("https://cdn.netkeiba.com/img.db//style/netkeiba.ja/image/review_bar_"))
        #芝適正(値が大きいほどダート適正)
        turf_type=soup_txt_review[1].get("width")
        #距離適性(値が大きいほど長距離適性)
        dist_type=soup_txt_review[3].get("width")
        #脚質(値が大きいほど追い込み)
        run_type=soup_txt_review[5].get("width")
        #成長(値が大きいほど晩成)
        grow_type=soup_txt_review[7].get("width")
        #馬場適性(値が大きいほど重馬場苦手)
        field_type=soup_txt_review[9].get("width")

        #馬の総合評価
        soup_stars=soup.find_all(class_=re.compile("star"))
        #総合評価
        try:
            soup_all_stars=soup_stars[0].contents[0].contents[0]
        except IndexError:
            soup_all_stars=""

        #実績評価
        try:
            soup_result_stars=soup_stars[1].contents[0]
        except IndexError:
            soup_result_stars=""
        #ポテンシャル評価
        try:
            soup_potential_stars=soup_stars[2].contents[0].contents[0]
        except IndexError:
            soup_potential_stars=""


        eachhorseInfo = [turf_type,dist_type,run_type,grow_type,field_type,soup_all_stars,soup_result_stars,soup_potential_stars]
        #過去3戦の結果格納
        for num in rank_result:
            eachhorseInfo.append(num)

        #過去3戦のオッズ格納
        for num in odds_result:
            eachhorseInfo.append(num)
        #不正な文字コードを削除
        eachhorseInfo_mod=[]
        for item in eachhorseInfo:
            item_mod=item.replace("\xa0","") 
            eachhorseInfo_mod.append(item_mod)
        #print(eachhorseInfo_mod)
        #CSVに書き出し
        year=2020-count
        filepass1="/Users/yamagamihiroki/arima2021/horseInfo/arimahorse{}_test.csv"
        filepass2=filepass1.format(year)
        with open(filepass2, 'a',newline='',encoding="SHIFT-JIS") as f:
            csv.writer(f).writerow(eachhorseInfo_mod)
    col_names=["芝適性","距離適性","脚質","成長","馬場適性","総合評価","実績評価",
               "ポテンシャル評価","前走成績","前々走成績","前々前走成績","前走オッズ","前々走オッズ","前々前走オッズ"]
    df=pandas.read_csv(filepass2,encoding="SHIFT-JIS",names=col_names)
    year=2020-count
    filepass3="/Users/yamagamihiroki/arima2021/horseInfo/arimahorse{}.csv"
    filepass4=filepass3.format(year)
    #転置
    df.T.to_csv(filepass4)
    #testファイルの削除
    os.remove(filepass2)


FileExistsError: [Errno 17] File exists: '/Users/yamagamihiroki/arima2021/horseInfo/'

In [172]:
#レースデータと出走馬のデータの結合
import requests
from bs4 import BeautifulSoup
import csv
import re
import time
import pandas
import os

#ディレクトリ作成
os.makedirs("/Users/yamagamihiroki/arima2021/allInfo/")

for year in range(2001,2021):
    racepass="/Users/yamagamihiroki/arima2021/raceInfo/arima"+str(year)+".csv"
    horsepass="/Users/yamagamihiroki/arima2021/horseInfo/arimahorse"+str(year)+".csv"
    df1=pandas.read_csv(racepass)
    df2=pandas.read_csv(horsepass)
    df_concat=pandas.concat([df1,df2], axis=0, ignore_index=False)
    allInfopass_tmp="/Users/yamagamihiroki/arima2021/allInfo/arima{}.csv"
    allInfopass=allInfopass_tmp.format(year)
    df_concat.to_csv(allInfopass, index=False)

In [162]:
#確認(年の部分を変更する)
pandas.read_csv("/Users/yamagamihiroki/arima2021/horseInfo/arimahorse2009.csv",index_col=0)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
芝適性,6.00,5.00,5.00,8.00,7.00,12.0,8.00,6.00,9.00,20.00,9.00,11.00,11.0,15.00,9.00,7
距離適性,63.00,64.00,58.00,99.00,98.00,54.0,75.00,70.00,65.00,60.00,83.00,90.00,61.0,57.00,61.00,95
脚質,110.00,108.00,84.00,87.00,83.00,79.0,36.00,95.00,7.00,6.00,8.00,83.00,16.0,32.00,86.00,38
成長,70.00,48.00,76.00,81.00,93.00,49.0,78.00,66.00,77.00,46.00,88.00,85.00,53.0,43.00,44.00,83
馬場適性,61.00,53.00,62.00,51.00,57.00,45.0,45.00,71.00,28.00,46.00,28.00,49.00,42.0,24.00,64.00,67
総合評価,4.23,4.29,3.58,3.55,3.88,3.1,3.92,3.50,3.36,3.59,3.75,3.38,3.3,3.39,3.70,3.5
実績評価,4.23,4.29,3.58,3.55,3.88,3.1,3.92,3.50,3.36,3.59,3.75,3.38,3.3,3.39,3.70,3.5
ポテンシャル評価,4.23,4.37,3.42,3.55,3.97,3.1,4.00,3.33,3.64,3.45,3.81,3.50,3.2,3.28,3.58,3.6
前走成績,10.00,7.00,17.00,10.00,11.00,16.0,7.00,13.00,10.00,10.00,9.00,12.00,16.0,17.00,5.00,中
前々走成績,9.00,1.00,2.00,3.00,10.00,1.0,17.00,14.00,13.00,12.00,12.00,12.00,6.0,16.00,15.00,1


In [173]:
#CSV確認(年によって変える)
pandas.read_csv("/Users/yamagamihiroki/arima2021/allInfo/arima2020.csv",index_col=0)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
馬名,クロノジェネシス,サラキア,フィエールマン,ラッキーライラック,ワールドプレミア,カレンブーケドール,ペルシアンナイト,クレッシェンドラヴ,オセアグレイト,ラヴズオンリーユー,ユーキャンスマイル,キセキ,バビット,オーソリティ,モズベッロ,ブラストワンピース
騎手名,北村友一,松山弘平,ルメール,福永祐一,武豊,池添謙一,大野拓弥,坂井瑠星,横山典弘,Ｍ．デム,岩田康誠,浜中俊,内田博幸,川田将雅,田辺裕信,横山武史
枠順,9,14,13,7,5,10,8,3,15,4,16,6,1,12,11,2
走破タイム,2:35.0,2:35.0,2:35.1,2:35.5,2:35.6,2:35.6,2:35.6,2:35.8,2:35.9,2:35.9,2:36.4,2:36.5,2:36.8,2:37.0,2:38.2,NaN
オッズ,2.5,74.9,3.5,8.1,13.5,7.9,148.1,188.1,183.4,15.5,157.0,24.9,36.2,20.3,174.6,33.1
通過順位,12-12-3-3,13-13-13-12,2-4-2-2,9-9-7-5,5-5-10-12,8-5-3-3,9-9-10-9,5-5-7-9,5-5-7-8,9-9-10-9,15-14-16-15,14-14-6-5,1-1-1-1,2-2-3-5,15-16-14-14,2-2-14-16
馬体重,474(+10),448(-2),478(0),522(0),484(+2),478(+4),500(0),500(0),494(-2),480(-6),504(+6),516(+8),458(+8),514(+2),470(-10),546(-4)
性齢,牝4,牝5,牡5,牝5,牡4,牝4,牡6,牡6,牡4,牝4,牡5,牡6,牡3,牡3,牡4,牡5
斤量,55,55,57,55,57,55,57,57,57,55,57,57,55,55,57,57
上がり3ハロン,36.2,35.4,36.5,36.5,36.3,36.8,36.2,36.8,36.8,36.6,36.5,37.6,38.4,38.2,38.5,NaN
